# Метрики качества классификации

Данное задание основано на материалах лекций по метрикам качества классификации.

## Вы научитесь:
• вычислять различные меры качества классификации: долю правильных ответов, точность, полноту, AUC-ROC и т.д.

• сравнивать алгоритмы классификации при наличии ограничений на точность или полноту

## Введение

В задачах классификации может быть много особенностей, влияющих на подсчет качества: различные цены ошибок, несбалансированность классов и т.д. Из-за этого существует большое количество метрик качества — каждая из них рассчитана на определенное сочетание свойств задачи и требований к ее решению.

Меры качества классификации можно разбить на две большие группы: предназначенные для алгоритмов, выдающих номера классов, и для алгоритмов, выдающих оценки принадлежности к классам. К первой группе относятся доля правильных ответов, точность, полнота, F-мера. Ко второй — площади под ROC- или PR-кривой.

## Реализация в Sklearn

Загрузим файл classification.csv.В нем записаны истинные классы объектов выборки (колонка true) и ответы некоторого классификатора (колонка predicted).

In [147]:
import pandas as pd
df = pd.read_csv("classification.csv")
eq = df.query('true == pred') #TP и TN
nq = df.query('true != pred') #FP и FN

Заполним таблицу ошибок классификации:

In [146]:
TP = eq.query('true == 1')
TN = eq.query('true == 0')
FP = nq.query('true == 0')
FN = nq.query('true == 1')
d = [
    [TP.count()[0], FN.count()[0]],
    [FP.count()[0], TN.count()[0]]
]
pd.DataFrame(data=d,columns=['Actual Positive','Actual Negative'],index=['Predicted Positive', 'Predicted Negative'])

,Actual Positive,Actual Negative
Predicted Positive,43,59
Predicted Negative,34,64


Используя Sklearn, посчитаем основные метрики качества классификатора:

• Accuracy (доля верно угаданных) — sklearn.metrics.accuracy_score

• Precision (точность) — sklearn.metrics.precision_score

• Recall (полнота) — sklearn.metrics.recall_score

• F-мера — sklearn.metrics.f1_score

In [145]:
from sklearn import metrics

accuracy = metrics.accuracy_score(y_true=df['true'],y_pred=df['pred'])
precision = metrics.precision_score(y_true=df['true'],y_pred=df['pred'])
recall = metrics.recall_score(y_true=df['true'],y_pred=df['pred'])
f1 = metrics.f1_score(y_true=df['true'],y_pred=df['pred'])

print("Accuracy: {}\nPrecision: {}\nRecall: {}\nF-мера: {}".format(accuracy,precision,recall,f1))

Accuracy: 0.535
Precision: 0.5584415584415584
Recall: 0.4215686274509804
F-мера: 0.48044692737430167


Имеется четыре обученных классификатора. В файле scores.csv записаны истинные классы и значения степени принадлежности положительному классу для каждого классификатора на некоторой выборке:

• для логистической регрессии — вероятность положительного класса (колонка score_logreg),

• для SVM — отступ от разделяющей поверхности (колонка score_svm),

• для метрического алгоритма — взвешенная сумма классов соседей (колонка score_knn),

• для решающего дерева — доля положительных объектов в листе (колонка score_tree).

Загрузим этот файл:

In [68]:
df_score = pd.read_csv('scores.csv')
target = df_score['true']
data_logreg = df_score['score_logreg']
data_svm = df_score['score_svm']
data_knn = df_score['score_knn']
data_tree = df_score['score_tree']

Рассчитаем площадь под ROC-кривой для каждого классификатора и найдём среди них наибольшее значение метрики AUC-ROC:

In [143]:
roc = {}
roc['logreg'] = metrics.roc_auc_score(y_true=target, y_score=data_logreg)
roc['svm'] = metrics.roc_auc_score(y_true=target, y_score=data_svm)
roc['knn'] = metrics.roc_auc_score(y_true=target, y_score=data_knn)
roc['tree'] = metrics.roc_auc_score(y_true=target, y_score=data_tree)

df_roc = pd.DataFrame(data=roc,index=['roc_auc_score']).transpose()
df_roc.sort_values(['roc_auc_score'],ascending=False,inplace=True)
df_roc.head(1)

,roc_auc_score
logreg,0.719188


Рассчитаем, какой классификатор достигает наибольшей точности (Precision) при полноте (Recall) не менее 70%:

In [144]:
from sklearn.metrics import precision_recall_curve

def calculate_precision(y_true=None, y_score=None):
    prec, recall, threshold = precision_recall_curve(y_true, y_score)
    dt = pd.DataFrame(prec,columns=['precision'])
    dt['recall'] = recall
    return dt.query('recall > 0.7').max()
acc = {}
acc['logreg']=calculate_precision(y_true=target, y_score=data_logreg)
acc['svm']=calculate_precision(y_true=target, y_score=data_svm)
acc['knn']=calculate_precision(y_true=target, y_score=data_knn)
acc['tree']=calculate_precision(y_true=target, y_score=data_tree)

df_prec = pd.DataFrame(data=acc).transpose()
df_prec.sort_values(['precision'],ascending=False,inplace=True)
df_prec.head(1)

,precision,recall
tree,0.651786,1.0
